In [ ]:
#@title 1.1 Instalar Dependencias { display-mode: "form" }

%%capture
!pip install pandas openpyxl xlsxwriter tqdm xlrd chardet scikit-learn numpy

print("✅ Dependencias instaladas correctamente")

In [ ]:
#@title 1.2 Importar Librerías { display-mode: "form" }

import pandas as pd
import numpy as np
import re
import warnings
import os
import gc
import io
import zipfile
import chardet
from datetime import datetime
from typing import Tuple, Optional, List, Dict, Any
from tqdm.notebook import tqdm
from google.colab import files
from IPython.display import display, HTML

# Machine Learning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 60)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

print("="*70)
print("🧠 ETL CONSOLIDADO T25 - EDICIÓN MACHINE LEARNING")
print("="*70)
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("✅ Librerías cargadas correctamente")

🧠 ETL CONSOLIDADO T25 - EDICIÓN MACHINE LEARNING
📅 2025-12-11 14:15:58
✅ Librerías cargadas correctamente


In [ ]:
#@title 2.1 Clasificador Inteligente de Texto { display-mode: "form" }

class ClasificadorTextoMedico:
    """
    Clasificador ML para detectar si un texto es:
    - Manual tarifario (SOAT, ISS, PROPIO)
    - Descripción de procedimiento médico
    - Porcentaje o valor numérico
    """

    # Vocabulario de referencia para manuales tarifarios
    VOCABULARIO_MANUAL = [
        'SOAT', 'SOAT VIGENTE', 'SOAT UVT', 'SOAT UVB', 'TARIFARIO SOAT',
        'ISS', 'ISS 2001', 'ISS2001', 'TARIFARIOS ISS', 'TARIFA ISS',
        'TARIFA PROPIA', 'TARIFAS PROPIAS', 'PROPIA', 'PROPIO', 'PROPIAS',
        'INSTITUCIONAL', 'TARIFAS INSTITUCIONALES', 'TARIFA INSTITUCIONAL',
        'DECRETO 2423', 'DECRETO 2644', 'UVT', 'UVB', 'TARIFA PLENA',
        'MENOS', 'PLENO', 'VIGENTE', 'MANUAL TARIFARIO'
    ]

    # Vocabulario de procedimientos médicos
    VOCABULARIO_MEDICO = [
        'CONSULTA', 'TERAPIA', 'NEURAL', 'CIRUGIA', 'PROCEDIMIENTO',
        'TRATAMIENTO', 'EVALUACION', 'VALORACION', 'DIAGNOSTICO',
        'EXAMEN', 'BIOPSIA', 'ECOGRAFIA', 'RADIOGRAFIA', 'TOMOGRAFIA',
        'RESONANCIA', 'LABORATORIO', 'HEMOGRAMA', 'CURACION', 'SUTURA',
        'INYECCION', 'APLICACION', 'NEBULIZACION', 'HOSPITALIZACION',
        'CONTROL', 'SEGUIMIENTO', 'ESPECIALISTA', 'MEDICINA', 'GENERAL',
        'PEDIATRIA', 'GINECOLOGIA', 'ORTOPEDIA', 'CARDIOLOGIA', 'NEUROLOGIA',
        'PSIQUIATRIA', 'PSICOLOGIA', 'FISIOTERAPIA', 'FONOAUDIOLOGIA',
        'ODONTOLOGIA', 'OPTOMETRIA', 'ANESTESIA', 'URGENCIA', 'AMBULANCIA',
        'SANGRE', 'ORINA', 'GLUCOSA', 'COLESTEROL', 'TRIGLICERIDOS',
        'ELECTROCARDIOGRAMA', 'ENDOSCOPIA', 'COLONOSCOPIA', 'MAMOGRAFIA',
        'QUIMIOTERAPIA', 'RADIOTERAPIA', 'DIALISIS', 'TRASPLANTE', 'PROTESIS',
        'IMPLANTE', 'REHABILITACION', 'TERAPIA OCUPACIONAL', 'TERAPIA FISICA',
        'CONSULTA DE', 'VISITA DE', 'ATENCION DE', 'SERVICIO DE'
    ]

    def __init__(self):
        """Inicializa el clasificador y entrena el vectorizador."""
        # Crear corpus de entrenamiento
        self.corpus_manual = self.VOCABULARIO_MANUAL
        self.corpus_medico = self.VOCABULARIO_MEDICO

        # Entrenar vectorizador TF-IDF
        self.vectorizer = TfidfVectorizer(
            analyzer='char_wb',
            ngram_range=(2, 4),
            lowercase=True,
            max_features=1000
        )

        # Entrenar con todo el vocabulario
        todo_corpus = self.corpus_manual + self.corpus_medico
        self.vectorizer.fit(todo_corpus)

        # Vectores de referencia
        self.vec_manual = self.vectorizer.transform(self.corpus_manual)
        self.vec_medico = self.vectorizer.transform(self.corpus_medico)

        # Centroide de cada clase
        self.centroide_manual = np.asarray(self.vec_manual.mean(axis=0)).flatten()
        self.centroide_medico = np.asarray(self.vec_medico.mean(axis=0)).flatten()

        print("✅ Clasificador ML entrenado")
        print(f"   • Vocabulario manual: {len(self.corpus_manual)} términos")
        print(f"   • Vocabulario médico: {len(self.corpus_medico)} términos")

    def clasificar(self, texto: str) -> Dict[str, Any]:
        """
        Clasifica un texto y retorna probabilidades.

        Returns:
            Dict con:
            - 'tipo': 'MANUAL', 'MEDICO', 'PORCENTAJE', 'DESCONOCIDO'
            - 'confianza': 0.0 a 1.0
            - 'scores': diccionario con scores de cada clase
        """
        if pd.isna(texto) or str(texto).strip() == '':
            return {'tipo': 'VACIO', 'confianza': 1.0, 'scores': {}}

        texto = str(texto).strip()
        texto_upper = texto.upper()

        # 1. Reglas rápidas basadas en patrones
        # ─────────────────────────────────────────────────────────────────────

        # Es un porcentaje o número
        if re.match(r'^[+-]?[\d,\.%\s]+$', texto):
            return {'tipo': 'PORCENTAJE', 'confianza': 0.95, 'scores': {'porcentaje': 0.95}}

        # Contiene palabras clave de manual tarifario
        palabras_manual = ['SOAT', 'ISS', 'TARIFA', 'DECRETO', 'UVT', 'UVB', 'PROPIA', 'PROPIO', 'INSTITUCIONAL']
        for palabra in palabras_manual:
            if palabra in texto_upper:
                return {'tipo': 'MANUAL', 'confianza': 0.9, 'scores': {'manual': 0.9}}

        # Contiene palabras clave médicas
        palabras_medicas = ['CONSULTA', 'TERAPIA', 'CIRUGIA', 'PROCEDIMIENTO', 'EXAMEN',
                           'TRATAMIENTO', 'BIOPSIA', 'ECOGRAFIA', 'LABORATORIO']
        for palabra in palabras_medicas:
            if palabra in texto_upper:
                return {'tipo': 'MEDICO', 'confianza': 0.85, 'scores': {'medico': 0.85}}

        # 2. Clasificación ML con TF-IDF
        # ─────────────────────────────────────────────────────────────────────
        try:
            vec_texto = self.vectorizer.transform([texto_upper])
            vec_array = np.asarray(vec_texto.todense()).flatten()

            # Calcular similitud con centroides
            sim_manual = cosine_similarity([vec_array], [self.centroide_manual])[0][0]
            sim_medico = cosine_similarity([vec_array], [self.centroide_medico])[0][0]

            # Normalizar scores
            total = sim_manual + sim_medico + 0.001
            score_manual = sim_manual / total
            score_medico = sim_medico / total

            scores = {'manual': score_manual, 'medico': score_medico}

            if score_manual > score_medico and score_manual > 0.4:
                return {'tipo': 'MANUAL', 'confianza': score_manual, 'scores': scores}
            elif score_medico > score_manual and score_medico > 0.4:
                return {'tipo': 'MEDICO', 'confianza': score_medico, 'scores': scores}
            else:
                return {'tipo': 'DESCONOCIDO', 'confianza': max(score_manual, score_medico), 'scores': scores}

        except Exception as e:
            return {'tipo': 'ERROR', 'confianza': 0.0, 'scores': {}, 'error': str(e)}

    def es_descripcion_medica(self, texto: str) -> Tuple[bool, float]:
        """
        Verifica si un texto parece ser una descripción médica.

        Returns:
            (es_medico, confianza)
        """
        resultado = self.clasificar(texto)
        return resultado['tipo'] == 'MEDICO', resultado['confianza']

    def es_manual_tarifario(self, texto: str) -> Tuple[bool, float]:
        """
        Verifica si un texto parece ser un manual tarifario válido.

        Returns:
            (es_manual, confianza)
        """
        resultado = self.clasificar(texto)
        return resultado['tipo'] == 'MANUAL', resultado['confianza']


# Crear instancia global
clasificador_ml = ClasificadorTextoMedico()

✅ Clasificador ML entrenado
   • Vocabulario manual: 27 términos
   • Vocabulario médico: 64 términos


In [ ]:
#@title 2.2 Probar el Clasificador ML { display-mode: "form" }
#@markdown ### 🧪 **Prueba el clasificador con ejemplos**

print("="*70)
print("🧪 PRUEBAS DEL CLASIFICADOR ML")
print("="*70)

# Ejemplos de prueba
ejemplos = [
    # Manuales tarifarios
    "SOAT VIGENTE",
    "TARIFA PROPIA",
    "ISS 2001 + 80%",
    "DECRETO 2644 UVT",
    "TARIFAS INSTITUCIONALES",

    # Descripciones médicas
    "CONSULTA DE MEDICINA GENERAL",
    "TERAPIA NEURAL POR SESION",
    "CIRUGIA DE RODILLA",
    "ECOGRAFIA ABDOMINAL",
    "LABORATORIO HEMOGRAMA",

    # Porcentajes
    "-30%",
    "0.05",
    "ISS + 65%",

    # Casos ambiguos
    "CONSULTA",
    "PROPIA",
    "VALORACION"
]

print(f"\n{'Texto':<45} {'Tipo':<12} {'Confianza':<10}")
print("─"*70)

for ejemplo in ejemplos:
    resultado = clasificador_ml.clasificar(ejemplo)
    print(f"{ejemplo:<45} {resultado['tipo']:<12} {resultado['confianza']:.2f}")

🧪 PRUEBAS DEL CLASIFICADOR ML

Texto                                         Tipo         Confianza 
──────────────────────────────────────────────────────────────────────
SOAT VIGENTE                                  MANUAL       0.90
TARIFA PROPIA                                 MANUAL       0.90
ISS 2001 + 80%                                MANUAL       0.90
DECRETO 2644 UVT                              MANUAL       0.90
TARIFAS INSTITUCIONALES                       MANUAL       0.90
CONSULTA DE MEDICINA GENERAL                  MEDICO       0.85
TERAPIA NEURAL POR SESION                     MEDICO       0.85
CIRUGIA DE RODILLA                            MEDICO       0.85
ECOGRAFIA ABDOMINAL                           MEDICO       0.85
LABORATORIO HEMOGRAMA                         MEDICO       0.85
-30%                                          PORCENTAJE   0.95
0.05                                          PORCENTAJE   0.95
ISS + 65%                                     MANUAL       0

In [ ]:
#@title 3.1 Clase ETL con Machine Learning { display-mode: "form" }

class ETLConsolidadoT25_ML:
    """
    ═══════════════════════════════════════════════════════════════════════════
    ETL CONSOLIDADO T25 - VERSIÓN CON MACHINE LEARNING
    ═══════════════════════════════════════════════════════════════════════════
    Sistema inteligente que detecta y corrige automáticamente cuando:
    - manual_tarifario contiene descripciones médicas
    - porcentaje_manual_tarifario contiene el manual real
    - Los valores están intercambiados entre columnas
    ═══════════════════════════════════════════════════════════════════════════
    """

    ANOS_IGNORAR = {'1996', '2001', '2016', '2022', '2023', '2024', '2025', '2644', '2423', '780'}

    PATRON_NUMERO = re.compile(r'[+-]?\d+(?:[,\.]\d+)?')
    PATRON_PORCENTAJE_FINAL = re.compile(r'[+-]\s*(\d+(?:[,\.]\d+)?)\s*$')
    PATRON_MENOS = re.compile(r'MENOS\s*(\d+(?:[,\.]\d+)?)', re.IGNORECASE)
    PATRON_MAS = re.compile(r'(?:MAS|\+)\s*(\d+(?:[,\.]\d+)?)', re.IGNORECASE)
    PATRON_DECIMAL = re.compile(r'^(-?0\.\d+)$')

    def __init__(self, clasificador: ClasificadorTextoMedico, chunk_size: int = 50000):
        """Inicializa el ETL con el clasificador ML."""
        self.clasificador = clasificador
        self.chunk_size = chunk_size
        self.stats = {
            'total_registros': 0,
            'columnas_intercambiadas': 0,
            'manuales_normalizados': 0,
            'porcentajes_extraidos': 0,
            'anomalias_detectadas': [],
            'correcciones_ml': []
        }
        self.resultados = {}

    def _detectar_y_corregir_anomalia(self, row: pd.Series) -> Dict[str, Any]:
        """
        Detecta si hay anomalía en la fila y sugiere corrección.

        Anomalías detectadas:
        1. manual_tarifario tiene descripción médica
        2. porcentaje_manual_tarifario tiene el manual real
        3. Valores intercambiados

        Returns:
            Dict con correcciones sugeridas
        """
        manual = str(row.get('manual_tarifario', '')).strip()
        porcentaje = str(row.get('porcentaje_manual_tarifario', '')).strip()
        descripcion = str(row.get('descripcion_del_cups', '')).strip()

        correccion = {
            'necesita_correccion': False,
            'nuevo_manual': manual,
            'nuevo_porcentaje': porcentaje,
            'razon': None,
            'confianza': 0.0
        }

        if not manual:
            return correccion

        # Clasificar el contenido de manual_tarifario
        clasif_manual = self.clasificador.clasificar(manual)
        clasif_porcentaje = self.clasificador.clasificar(porcentaje)

        # CASO 1: manual_tarifario tiene descripción médica
        if clasif_manual['tipo'] == 'MEDICO' and clasif_manual['confianza'] > 0.6:
            # Verificar si porcentaje tiene el manual real
            if clasif_porcentaje['tipo'] == 'MANUAL' and clasif_porcentaje['confianza'] > 0.5:
                correccion['necesita_correccion'] = True
                correccion['nuevo_manual'] = porcentaje
                correccion['nuevo_porcentaje'] = '0'  # Extraer del nuevo manual si hay
                correccion['razon'] = f"ML detectó descripción médica en manual_tarifario (conf: {clasif_manual['confianza']:.2f})"
                correccion['confianza'] = clasif_manual['confianza']
                return correccion

        # CASO 2: manual_tarifario es similar a descripcion_del_cups
        if descripcion and len(manual) > 20:
            manual_words = set(manual.upper().split())
            desc_words = set(descripcion.upper().split())
            if len(manual_words) > 0 and len(desc_words) > 0:
                similitud = len(manual_words & desc_words) / min(len(manual_words), len(desc_words))
                if similitud > 0.5:  # Más del 50% de palabras en común
                    if clasif_porcentaje['tipo'] == 'MANUAL':
                        correccion['necesita_correccion'] = True
                        correccion['nuevo_manual'] = porcentaje
                        correccion['nuevo_porcentaje'] = '0'
                        correccion['razon'] = f"manual_tarifario similar a descripción ({similitud:.0%})"
                        correccion['confianza'] = similitud
                        return correccion

        # CASO 3: manual_tarifario tiene formato de tarifa (número grande)
        try:
            valor_manual = float(manual.replace(',', '.').replace('$', '').strip())
            if valor_manual > 1000:  # Parece una tarifa, no un manual
                if clasif_porcentaje['tipo'] == 'MANUAL':
                    correccion['necesita_correccion'] = True
                    correccion['nuevo_manual'] = porcentaje
                    correccion['nuevo_porcentaje'] = '0'
                    correccion['razon'] = f"manual_tarifario contiene tarifa ({valor_manual:,.0f})"
                    correccion['confianza'] = 0.9
                    return correccion
        except:
            pass

        return correccion

    def _extraer_porcentaje(self, texto: str) -> Optional[float]:
        """Extrae porcentaje de forma inteligente."""
        if pd.isna(texto) or str(texto).strip() == '':
            return None

        texto = str(texto).strip()
        texto_upper = texto.upper()

        if 'PLENA' in texto_upper or 'PLENO' in texto_upper:
            if not re.search(r'[+-]\s*\d+', texto_upper):
                return 0.0

        texto_sin_pct = texto.replace('%', '')

        match = self.PATRON_PORCENTAJE_FINAL.search(texto_sin_pct)
        if match:
            try:
                valor = float(match.group(1).replace(',', '.'))
                if '-' in texto_sin_pct:
                    valor = -abs(valor)
                if -100 <= valor <= 200:
                    return valor
            except:
                pass

        match = self.PATRON_MENOS.search(texto_sin_pct)
        if match:
            try:
                return -float(match.group(1).replace(',', '.'))
            except:
                pass

        match = self.PATRON_MAS.search(texto_sin_pct)
        if match:
            try:
                return float(match.group(1).replace(',', '.'))
            except:
                pass

        match = self.PATRON_DECIMAL.match(texto_sin_pct.strip())
        if match:
            try:
                return round(float(match.group(1)) * 100, 2)
            except:
                pass

        numeros = self.PATRON_NUMERO.findall(texto_sin_pct)
        for num_str in reversed(numeros):
            try:
                num = float(num_str.replace(',', '.'))
                if str(int(abs(num))) in self.ANOS_IGNORAR:
                    continue
                if num > 1000:
                    continue
                if -100 <= num <= 200:
                    if 'MENOS' in texto_upper or f'-{num_str}' in texto:
                        num = -abs(num)
                    return num
            except:
                continue

        return None

    def _normalizar_manual(self, texto: str) -> str:
        """Normaliza el manual tarifario."""
        if pd.isna(texto) or str(texto).strip() == '':
            return 'PROPIO'

        texto = str(texto).strip()
        texto_upper = texto.upper()

        # PROPIO
        if re.search(r'\bPROPIA?S?\b|INSTITUCIONAL|TARIA\s*PROPIA', texto_upper):
            return 'PROPIO'

        # ISS
        if re.search(r'\bISS\b', texto_upper) and not re.search(r'\bSOAT\b', texto_upper):
            return 'ISS'

        # SOAT
        if re.search(r'\bSOAT\b|\bUVT\b|\bUVB\b|DECRETO\s*2423|DECRETO\s*2644', texto_upper):
            return 'SOAT'

        # Números puros -> PROPIO
        if re.match(r'^[\d,\.\s]+$', texto):
            return 'PROPIO'

        return texto

    def _procesar_fila(self, row: pd.Series) -> Dict[str, Any]:
        """
        Procesa una fila completa con detección ML.
        """
        resultado = {
            'manual_tarifario': '',
            'porcentaje_manual_tarifario': 0.0,
            'correccion_aplicada': False,
            'log': None
        }

        # 1. Detectar anomalías
        correccion = self._detectar_y_corregir_anomalia(row)

        if correccion['necesita_correccion']:
            # Usar valores corregidos
            manual_raw = correccion['nuevo_manual']
            porcentaje_raw = correccion['nuevo_porcentaje']
            resultado['correccion_aplicada'] = True
            resultado['log'] = correccion['razon']
        else:
            manual_raw = str(row.get('manual_tarifario', '')).strip()
            porcentaje_raw = str(row.get('porcentaje_manual_tarifario', '')).strip()

        # 2. Normalizar manual
        resultado['manual_tarifario'] = self._normalizar_manual(manual_raw)

        # 3. Extraer porcentaje
        tarifa = row.get('tarifa_unitaria_en_pesos', '0')
        try:
            tarifa_num = float(str(tarifa).replace(',', '.'))
        except:
            tarifa_num = 0

        # Si porcentaje es texto de manual o propio -> 0
        porcentaje_upper = porcentaje_raw.upper()
        if any(p in porcentaje_upper for p in ['PROPIO', 'PROPIA', 'INSTITUCIONAL', 'PLENA', 'PLENO']):
            resultado['porcentaje_manual_tarifario'] = 0.0
        else:
            # Extraer porcentaje
            pct = self._extraer_porcentaje(porcentaje_raw)
            if pct is not None:
                # Verificar que no sea igual a la tarifa
                if tarifa_num > 0 and abs(pct - tarifa_num) < 1:
                    resultado['porcentaje_manual_tarifario'] = 0.0
                elif pct > 1000:  # Probable tarifa duplicada
                    resultado['porcentaje_manual_tarifario'] = 0.0
                else:
                    resultado['porcentaje_manual_tarifario'] = round(pct, 2)
            else:
                resultado['porcentaje_manual_tarifario'] = 0.0

        return resultado

    def procesar_dataframe(self, df: pd.DataFrame, nombre: str = "Datos") -> pd.DataFrame:
        """Procesa un DataFrame completo."""
        print(f"\n{'═'*70}")
        print(f"🧠 PROCESANDO CON ML: {nombre}")
        print(f"{'═'*70}")

        inicio = datetime.now()
        total = len(df)
        print(f"📊 Total registros: {total:,}")

        # Normalizar columnas
        df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

        # Asegurar columnas
        for col in ['manual_tarifario', 'porcentaje_manual_tarifario', 'tarifa_unitaria_en_pesos']:
            if col not in df.columns:
                df[col] = ''

        # Procesar
        print(f"\n🔄 Fase 1: Detección de anomalías con ML...")

        nuevos_manuales = []
        nuevos_porcentajes = []
        correcciones = []

        for idx in tqdm(range(total), desc="   Procesando"):
            row = df.iloc[idx]
            resultado = self._procesar_fila(row)

            nuevos_manuales.append(resultado['manual_tarifario'])
            nuevos_porcentajes.append(resultado['porcentaje_manual_tarifario'])

            if resultado['correccion_aplicada']:
                correcciones.append({
                    'indice': idx,
                    'original_manual': row.get('manual_tarifario', ''),
                    'original_porcentaje': row.get('porcentaje_manual_tarifario', ''),
                    'nuevo_manual': resultado['manual_tarifario'],
                    'log': resultado['log']
                })

        df['manual_tarifario'] = nuevos_manuales
        df['porcentaje_manual_tarifario'] = nuevos_porcentajes

        # Corregir tarifas
        print(f"\n🔄 Fase 2: Corrigiendo tarifas...")
        tarifa = pd.to_numeric(
            df['tarifa_unitaria_en_pesos'].astype(str).str.replace(',', '.'),
            errors='coerce'
        ).fillna(0)
        mask_pequena = (tarifa > 0) & (tarifa < 100)
        tarifa.loc[mask_pequena] = tarifa.loc[mask_pequena] * 1000
        df['tarifa_unitaria_en_pesos'] = tarifa.round(2)

        # Estadísticas
        duracion = (datetime.now() - inicio).total_seconds()

        print(f"\n{'─'*70}")
        print(f"📈 RESULTADOS - {nombre}")
        print(f"{'─'*70}")
        print(f"⏱️ Tiempo: {duracion:.1f} segundos")
        print(f"🔧 Correcciones ML aplicadas: {len(correcciones):,}")

        if correcciones:
            print(f"\n📝 Muestra de correcciones aplicadas:")
            for c in correcciones[:10]:
                print(f"   • Fila {c['indice']}: {c['log']}")
                print(f"     Original: '{str(c['original_manual'])[:40]}...'")
                print(f"     Corregido: '{c['nuevo_manual']}'")

        print(f"\n📊 Distribución de Manuales:")
        for manual, count in df['manual_tarifario'].value_counts().head(10).items():
            pct = count / total * 100
            print(f"   {manual:15} │ {count:>10,} │ {pct:5.1f}%")

        self.stats['total_registros'] += total
        self.stats['columnas_intercambiadas'] += len(correcciones)
        self.stats['correcciones_ml'].extend(correcciones)

        gc.collect()
        return df

    def ejecutar(self, contenido: bytes, nombre: str) -> Dict[str, pd.DataFrame]:
        """Ejecuta el ETL completo."""
        print("\n" + "═"*70)
        print("🚀 ETL CONSOLIDADO T25 - ML EDITION")
        print("═"*70)
        print(f"📁 Archivo: {nombre}")
        print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

        inicio = datetime.now()

        # Cargar archivo
        dataframes = self._cargar_archivo(contenido, nombre)

        # Procesar cada hoja
        for nombre_hoja, df in dataframes.items():
            self.resultados[nombre_hoja] = self.procesar_dataframe(df, nombre_hoja)

        duracion = (datetime.now() - inicio).total_seconds()

        print("\n" + "═"*70)
        print("✅ ETL ML COMPLETADO")
        print("═"*70)
        print(f"📊 Total registros: {self.stats['total_registros']:,}")
        print(f"🔧 Correcciones ML: {self.stats['columnas_intercambiadas']:,}")
        print(f"⏱️ Tiempo: {duracion:.1f} segundos")

        return self.resultados

    def _cargar_archivo(self, contenido: bytes, nombre: str) -> Dict[str, pd.DataFrame]:
        """Carga archivo Excel o CSV."""
        dataframes = {}

        if nombre.endswith('.csv'):
            resultado = chardet.detect(contenido[:10000])
            encoding = resultado['encoding'] or 'utf-8'

            for sep in [';', ',', '\t']:
                try:
                    df = pd.read_csv(io.BytesIO(contenido), sep=sep, encoding=encoding,
                                    dtype=str, low_memory=False)
                    if len(df.columns) > 1:
                        dataframes['Datos'] = df
                        break
                except:
                    continue
        else:
            excel = pd.ExcelFile(io.BytesIO(contenido))
            for hoja in excel.sheet_names:
                df = pd.read_excel(excel, sheet_name=hoja, dtype=str)
                if len(df) > 0:
                    dataframes[hoja] = df

        return dataframes

    def exportar_log_correcciones(self, archivo: str = 'correcciones_ml.csv'):
        """Exporta log de correcciones ML."""
        if self.stats['correcciones_ml']:
            df_log = pd.DataFrame(self.stats['correcciones_ml'])
            df_log.to_csv(archivo, index=False, encoding='utf-8-sig')
            print(f"✅ Log exportado: {archivo}")
            return df_log
        else:
            print("ℹ️ No hay correcciones ML para exportar")
            return None


print("✅ Clase ETLConsolidadoT25_ML definida")

✅ Clase ETLConsolidadoT25_ML definida


In [ ]:
#@title 4.1 Subir Archivo { display-mode: "form" }
#@markdown ### 📁 **Sube tu archivo Excel o CSV**

print("="*70)
print("📤 SUBIR ARCHIVO")
print("="*70)
print("\nFormatos soportados: .xlsx, .xls, .csv, .zip")
print()

uploaded = files.upload()

if uploaded:
    nombre_archivo = list(uploaded.keys())[0]
    contenido_archivo = uploaded[nombre_archivo]

    print(f"\n✅ Archivo cargado: {nombre_archivo}")
    print(f"📦 Tamaño: {len(contenido_archivo)/1024/1024:.2f} MB")

📤 SUBIR ARCHIVO

Formatos soportados: .xlsx, .xls, .csv, .zip



Saving CONSOLIDADO_COMPLETO_20251211_0207 - copia.xlsx to CONSOLIDADO_COMPLETO_20251211_0207 - copia (1).xlsx

✅ Archivo cargado: CONSOLIDADO_COMPLETO_20251211_0207 - copia (1).xlsx
📦 Tamaño: 66.20 MB


In [ ]:
#@title 4.2 Ejecutar ETL con Machine Learning { display-mode: "form" }
#@markdown ### ⚙️ **Procesa el archivo con detección inteligente de anomalías**

if 'contenido_archivo' not in dir():
    print("❌ ERROR: Primero sube un archivo")
else:
    # Crear ETL con clasificador ML
    etl_ml = ETLConsolidadoT25_ML(
        clasificador=clasificador_ml,
        chunk_size=50000
    )

    # Ejecutar
    dataframes_limpios = etl_ml.ejecutar(contenido_archivo, nombre_archivo)

    print("\n📦 DataFrames disponibles en: dataframes_limpios")


══════════════════════════════════════════════════════════════════════
🚀 ETL CONSOLIDADO T25 - ML EDITION
══════════════════════════════════════════════════════════════════════
📁 Archivo: CONSOLIDADO_COMPLETO_20251211_0207 - copia (1).xlsx
📅 2025-12-11 14:31:52

══════════════════════════════════════════════════════════════════════
🧠 PROCESANDO CON ML: CONSOLIDADO_1
══════════════════════════════════════════════════════════════════════
📊 Total registros: 1,000,000

🔄 Fase 1: Detección de anomalías con ML...


   Procesando:   0%|          | 0/1000000 [00:00<?, ?it/s]


🔄 Fase 2: Corrigiendo tarifas...

──────────────────────────────────────────────────────────────────────
📈 RESULTADOS - CONSOLIDADO_1
──────────────────────────────────────────────────────────────────────
⏱️ Tiempo: 288.7 segundos
🔧 Correcciones ML aplicadas: 389

📝 Muestra de correcciones aplicadas:
   • Fila 181898: ML detectó descripción médica en manual_tarifario (conf: 0.80)
     Original: 'nan...'
     Corregido: 'PROPIO'
   • Fila 181899: ML detectó descripción médica en manual_tarifario (conf: 0.80)
     Original: 'nan...'
     Corregido: 'PROPIO'
   • Fila 181900: ML detectó descripción médica en manual_tarifario (conf: 0.80)
     Original: 'nan...'
     Corregido: 'PROPIO'
   • Fila 181901: ML detectó descripción médica en manual_tarifario (conf: 0.80)
     Original: 'nan...'
     Corregido: 'PROPIO'
   • Fila 181902: ML detectó descripción médica en manual_tarifario (conf: 0.80)
     Original: 'nan...'
     Corregido: 'PROPIO'
   • Fila 181903: ML detectó descripción médica

   Procesando:   0%|          | 0/210497 [00:00<?, ?it/s]


🔄 Fase 2: Corrigiendo tarifas...

──────────────────────────────────────────────────────────────────────
📈 RESULTADOS - CONSOLIDADO_2
──────────────────────────────────────────────────────────────────────
⏱️ Tiempo: 57.7 segundos
🔧 Correcciones ML aplicadas: 1,449

📝 Muestra de correcciones aplicadas:
   • Fila 104782: ML detectó descripción médica en manual_tarifario (conf: 0.63)
     Original: 'ANEXO TÉCNICO EVENTO 2024...'
     Corregido: 'PROPIO'
   • Fila 104783: ML detectó descripción médica en manual_tarifario (conf: 0.63)
     Original: 'ANEXO TÉCNICO EVENTO 2024...'
     Corregido: 'PROPIO'
   • Fila 104784: ML detectó descripción médica en manual_tarifario (conf: 0.63)
     Original: 'ANEXO TÉCNICO EVENTO 2024...'
     Corregido: 'PROPIO'
   • Fila 104785: ML detectó descripción médica en manual_tarifario (conf: 0.63)
     Original: 'ANEXO TÉCNICO EVENTO 2024...'
     Corregido: 'PROPIO'
   • Fila 104786: ML detectó descripción médica en manual_tarifario (conf: 0.63)
     Or

In [ ]:
#@title 5.1 Ver Correcciones ML Aplicadas { display-mode: "form" }
#@markdown ### 🔧 **Muestra las correcciones detectadas por ML**

if 'etl_ml' not in dir():
    print("❌ ERROR: Primero ejecuta el ETL")
else:
    correcciones = etl_ml.stats['correcciones_ml']

    print("="*70)
    print("🔧 CORRECCIONES ML APLICADAS")
    print("="*70)
    print(f"\n📊 Total correcciones: {len(correcciones):,}")

    if correcciones:
        print(f"\n📝 Detalle de correcciones:")
        print("─"*70)

        for i, c in enumerate(correcciones[:50]):
            print(f"\n[{i+1}] Fila {c['indice']}")
            print(f"    📌 Razón: {c['log']}")
            print(f"    ❌ Original manual: {str(c['original_manual'])[:60]}")
            print(f"    ❌ Original %: {str(c['original_porcentaje'])[:40]}")
            print(f"    ✅ Nuevo manual: {c['nuevo_manual']}")

        if len(correcciones) > 50:
            print(f"\n... y {len(correcciones) - 50} correcciones más")
    else:
        print("\n✅ No se detectaron anomalías que requieran corrección ML")

🔧 CORRECCIONES ML APLICADAS

📊 Total correcciones: 1,838

📝 Detalle de correcciones:
──────────────────────────────────────────────────────────────────────

[1] Fila 181898
    📌 Razón: ML detectó descripción médica en manual_tarifario (conf: 0.80)
    ❌ Original manual: nan
    ❌ Original %: TARIFA PROPIA
    ✅ Nuevo manual: PROPIO

[2] Fila 181899
    📌 Razón: ML detectó descripción médica en manual_tarifario (conf: 0.80)
    ❌ Original manual: nan
    ❌ Original %: TARIFA PROPIA
    ✅ Nuevo manual: PROPIO

[3] Fila 181900
    📌 Razón: ML detectó descripción médica en manual_tarifario (conf: 0.80)
    ❌ Original manual: nan
    ❌ Original %: TARIFA PROPIA
    ✅ Nuevo manual: PROPIO

[4] Fila 181901
    📌 Razón: ML detectó descripción médica en manual_tarifario (conf: 0.80)
    ❌ Original manual: nan
    ❌ Original %: TARIFA PROPIA
    ✅ Nuevo manual: PROPIO

[5] Fila 181902
    📌 Razón: ML detectó descripción médica en manual_tarifario (conf: 0.80)
    ❌ Original manual: nan
    ❌ Or

In [ ]:
#@title 5.2 Ver Resumen de Resultados { display-mode: "form" }

if 'dataframes_limpios' not in dir():
    print("❌ ERROR: Primero ejecuta el ETL")
else:
    print("="*70)
    print("📊 RESUMEN DE RESULTADOS")
    print("="*70)

    for nombre, df in dataframes_limpios.items():
        print(f"\n{'─'*70}")
        print(f"📑 {nombre}: {len(df):,} registros")
        print(f"{'─'*70}")

        print(f"\n📋 Manuales Tarifarios:")
        for val, count in df['manual_tarifario'].value_counts().items():
            pct = count / len(df) * 100
            barra = '█' * int(pct / 2)
            print(f"   {val:15} │ {count:>10,} │ {pct:5.1f}% │ {barra}")

        print(f"\n📊 Top 10 Porcentajes:")
        for val, count in df['porcentaje_manual_tarifario'].value_counts().head(10).items():
            print(f"   {str(val):>10}% : {count:>10,}")

📊 RESUMEN DE RESULTADOS

──────────────────────────────────────────────────────────────────────
📑 CONSOLIDADO_1: 1,000,000 registros
──────────────────────────────────────────────────────────────────────

📋 Manuales Tarifarios:
   SOAT            │    584,470 │  58.4% │ █████████████████████████████
   PROPIO          │    296,693 │  29.7% │ ██████████████
   ISS             │    101,651 │  10.2% │ █████
   TARIFARIO PROPIO │     15,738 │   1.6% │ 
   Tercero + 5%    │        736 │   0.1% │ 
   ISS2001         │        418 │   0.0% │ 
   nan             │        245 │   0.0% │ 
   POR COTIZACIÓN  │         18 │   0.0% │ 
   TARIFAS POSITIVA │         10 │   0.0% │ 
   COTIZACION PREVIA │         10 │   0.0% │ 
   TARIFA POSITIVA │          5 │   0.0% │ 
   MINISTERIO      │          3 │   0.0% │ 
   PRECIO REGULADO │          2 │   0.0% │ 
   MOS 12%         │          1 │   0.0% │ 

📊 Top 10 Porcentajes:
          0.0% :    615,525
        -10.0% :     78,329
        -20.0% :     65,6

In [ ]:
#@title 5.3 Ver Muestra de Datos { display-mode: "form" }

#@markdown Filas a mostrar:
n_filas = 25 #@param {type:"slider", min:5, max:100, step:5}

if 'dataframes_limpios' not in dir():
    print("❌ ERROR: Primero ejecuta el ETL")
else:
    for nombre, df in dataframes_limpios.items():
        print(f"\n{'═'*70}")
        print(f"📑 MUESTRA: {nombre}")
        print(f"{'═'*70}")

        cols = ['codigo_cups', 'descripcion_del_cups', 'tarifa_unitaria_en_pesos',
                'manual_tarifario', 'porcentaje_manual_tarifario', 'contrato']
        cols_disp = [c for c in cols if c in df.columns]

        display(df[cols_disp].head(n_filas))


══════════════════════════════════════════════════════════════════════
📑 MUESTRA: CONSOLIDADO_1
══════════════════════════════════════════════════════════════════════


,codigo_cups,descripcion_del_cups,tarifa_unitaria_en_pesos,manual_tarifario,porcentaje_manual_tarifario,contrato
0,890280,CONSULTA DE PRIMERA VEZ POR ESPECIALISTA EN ORTOPEDIA Y ...,"75,400.00",SOAT,0.00,662-2024
1,890380,CONSULTA DE CONTROL O DE SEGUIMIENTO POR ESPECIALISTA EN...,"75,400.00",SOAT,0.00,662-2024
2,890502,PARTICIPACIÓN EN JUNTA MÉDICA O EQUIPO INTERDISCIPLINARI...,"150,436.00",PROPIO,0.00,662-2024
3,819101,ASPIRACIÓN ARTICULAR,"127,292.00",PROPIO,0.00,662-2024
4,819201,INYECCIÓN DE SUSTANCIA TERAPÉUTICA DENTRO DE LA ARTICULA...,"127,292.00",PROPIO,0.00,662-2024
5,819202,APLICACIÓN DE PLASMA RICO EN PLAQUETAS DENTRO DEL SISTEM...,"254,584.00",PROPIO,0.00,662-2024
6,770501,"SECUESTRECTOMÍA, DRENAJE, DESBRIDAMIENTO DE FÉMUR VÍA AB...","308,720.00",ISS,186.00,662-2024
7,770601,"SECUESTRECTOMÍA, DRENAJE, DESBRIDAMIENTO DE RÓTULA VÍA A...","199,760.00",ISS,186.00,662-2024
8,770701,"SECUESTRECTOMÍA, DRENAJE, DESBRIDAMIENTO DE TIBIA O PERONÉ","308,720.00",ISS,186.00,662-2024
9,770702,"SECUESTRECTOMÍA, DRENAJE, DESBRIDAMIENTO DE TIBIA Y PERONÉ","435,840.00",ISS,186.00,662-2024



══════════════════════════════════════════════════════════════════════
📑 MUESTRA: CONSOLIDADO_2
══════════════════════════════════════════════════════════════════════


,codigo_cups,descripcion_del_cups,tarifa_unitaria_en_pesos,manual_tarifario,porcentaje_manual_tarifario,contrato
0,385802,Oclusion. Pinzamiento O Ligadura Arterial Infrapatelar,"2,100,000.00",PROPIO,0.00,674-2024
1,385802,Oclusion. Pinzamiento O Ligadura Arterial Infrapatelar,"2,100,000.00",PROPIO,0.00,674-2024
2,385820,"Oclusion De Arterias De Miembros Inferiores, Via Percuta...","11,500,000.00",PROPIO,0.00,674-2024
3,385820,"Oclusion De Arterias De Miembros Inferiores, Via Percuta...","11,500,000.00",PROPIO,0.00,674-2024
4,385820,"Oclusion De Arterias De Miembros Inferiores, Via Percuta...","11,500,000.00",PROPIO,0.00,674-2024
5,385901,Oclusion. Pinzamiento O Ligadura Venosa Profunda Suprapa...,"3,807,813.10",PROPIO,0.00,674-2024
6,385901,Oclusion. Pinzamiento O Ligadura Venosa Profunda Suprapa...,"3,807,813.10",PROPIO,0.00,674-2024
7,385901,Oclusion. Pinzamiento O Ligadura Venosa Profunda Suprapa...,"3,807,813.10",PROPIO,0.00,674-2024
8,381102,Endarterectomia De Vasos Intracraneales,"13,500,000.00",PROPIO,0.00,674-2024
9,381102,Endarterectomia De Vasos Intracraneales,"13,500,000.00",PROPIO,0.00,674-2024


In [ ]:
#@title 6.1 Exportar a Excel { display-mode: "form" }

if 'dataframes_limpios' not in dir():
    print("❌ ERROR: Primero ejecuta el ETL")
else:
    print("="*70)
    print("💾 EXPORTANDO A EXCEL")
    print("="*70)

    nombre_base = nombre_archivo.rsplit('.', 1)[0]
    archivo_excel = f"{nombre_base}_ML_LIMPIO.xlsx"

    print(f"\n📝 Generando: {archivo_excel}")

    with pd.ExcelWriter(archivo_excel, engine='xlsxwriter') as writer:
        workbook = writer.book

        fmt_header = workbook.add_format({
            'bold': True, 'bg_color': '#6A1B9A', 'font_color': 'white',
            'align': 'center', 'border': 1
        })
        fmt_moneda = workbook.add_format({'num_format': '$#,##0.00'})

        for nombre_hoja, df in tqdm(dataframes_limpios.items(), desc="Exportando"):
            hoja = nombre_hoja[:31]
            df.to_excel(writer, sheet_name=hoja, index=False, startrow=1, header=False)

            ws = writer.sheets[hoja]
            for col, column in enumerate(df.columns):
                ws.write(0, col, column, fmt_header)
                if column == 'tarifa_unitaria_en_pesos':
                    ws.set_column(col, col, 20, fmt_moneda)
                else:
                    ws.set_column(col, col, 15)

            ws.freeze_panes(1, 0)
            ws.autofilter(0, 0, len(df), len(df.columns) - 1)

    print(f"\n✅ Excel generado: {archivo_excel}")

💾 EXPORTANDO A EXCEL

📝 Generando: CONSOLIDADO_COMPLETO_20251211_0207 - copia (1)_ML_LIMPIO.xlsx


Exportando:   0%|          | 0/2 [00:00<?, ?it/s]


✅ Excel generado: CONSOLIDADO_COMPLETO_20251211_0207 - copia (1)_ML_LIMPIO.xlsx


In [ ]:
#@title 6.3 Descargar Archivos { display-mode: "form" }

print("📥 Descargando archivos...")

if 'archivo_excel' in dir() and os.path.exists(archivo_excel):
    print(f"\n📊 Descargando Excel: {archivo_excel}")
    files.download(archivo_excel)

if os.path.exists('correcciones_ml.csv'):
    print(f"\n📝 Descargando log ML: correcciones_ml.csv")
    files.download('correcciones_ml.csv')

print("\n✅ Descargas iniciadas")

📥 Descargando archivos...

📊 Descargando Excel: CONSOLIDADO_COMPLETO_20251211_0207 - copia (1)_ML_LIMPIO.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Descargas iniciadas
